In [428]:
import pandas as pd
import numpy as np

from RBO import *
from getRecommendations import *
from dataPrep import *
from sklearn.metrics import *
from scipy.stats impo9rt linregress

from sklearn.metrics.pairwise import *

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [429]:
%%time
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
#prod_ing = prod_ing.sort_values(['id', 'order'])
df = dataPrep(ings, prods, prod_ing)

print('Number of products df: ', len(df))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products df:  15945
Number of unique ingredients:  6899
Number of total ingredients:  446618
Wall time: 7.23 s


In [364]:
%%time
##Long dataframes
df2_A = pd.read_csv('data/prodIng_Wide_01.csv', index_col=0)
df2_B = pd.read_csv('data/prodIng_Wide_02.csv', index_col=0)
df2_C = pd.read_csv('data/prodIng_Wide_03.csv', index_col=0)
frames = [df2_A, df2_B, df2_C]
df2 = pd.concat(frames)
df2 = df2.T

Wall time: 21.8 s


# GET RECOMMENDATIONS


In [486]:
product ='Facial Treatment Essence'

##get indices of all products
indices = pd.Series(df.index, index=df['product'])#.drop_duplicates()
##find index of specified product
idx = indices[product]

print('Product:', df.iloc[idx]['product'])
print('Brand:', df.iloc[idx]['brand'])
print('Id:', idx)
print('Number of ingredients:', df.iloc[idx]['ingCount'])
print('Ingredients', df.iloc[idx]['ingList'])

Product: Facial Treatment Essence
Brand: SK-II
Id: 1961
Number of ingredients: 7
Ingredients ['Saccharomyces Ferment Filtrate', 'Butylene Glycol', 'Pentylene Glycol', 'Water', 'Sodium Benzoate', 'Methylparaben', 'Sorbic Acid']


#### Find product id with name

In [440]:
product_find = 'Intensive Healing Cream Daily Fragrance-Free Cream'
idx_2 = indices[product_find]
#idReturn = row.index.item()

print('Product:', df.iloc[idx_2]['product'])
print('Brand:', df.iloc[idx_2]['brand'])
print('Id:', idx_2)
print('Number of ingredients:', df.iloc[idx_2]['ingCount'])
print('Ingredients', df.iloc[idx_2]['ingList'])

Product: Intensive Healing Cream Daily Fragrance-Free Cream
Brand: Curel
Id: 3671
Number of ingredients: 21
Ingredients ['Water', 'Mineral Oil', 'Petrolatum', 'Glycerin', 'Microcrystalline Wax', 'Cetyl PEG/PPG-10/1 Dimethicone', 'Hydrogenated Castor Oil', 'Glyceryl Dilaurate', 'Paraffin', 'Dimethicone', 'Cetyl-PG Hydroxyethyl Palmitamide', 'Magnesium Stearate', 'Isopropyl Myristate', 'Magnesium Sulfate', 'Glyceryl Oleate', 'Ethoxydiglycol', 'DMDM Hydantoin', 'Methylparaben', 'Butylene Glycol', 'Eucalyptus Globulus Leaf Extract', 'Propylparaben']


In [312]:
idx_2

4350

## Cosine similarity

Reference [here](https://www.datacamp.com/community/tutorials/recommender-systems-python)

### Create dataframe of top ten similar products

In [450]:
%%time
column = 'product'
# Get the index of the product that matches the product name
df['ing#List'] = df['ing#List'].astype(str)
tfidf = TfidfVectorizer(binary = True, stop_words = None)
tfidf_matrix = tfidf.fit_transform(df['ing#List'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df.index, index=df[column]).drop_duplicates()
idx = indices[product]

# Get the pairwsie similarity scores of all products with that product
sim_scores_all = list(enumerate(cosine_sim[idx]))

# Sort the products based on the similarity scores
sim_scores_sorted = sorted(sim_scores_all, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores_ten = sim_scores_sorted[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores_ten]

# Return the top 10 most similar products and their ingredients
#return df[['product', 'brand','ingList']].iloc[product_indices]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

Wall time: 33.6 s


### Show top ten

In [451]:
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
#recommended.to_csv('recommendations/COS.csv')
topTen.reset_index()

,index,product,brand,ingList,commonIng
0,2927,The Oil Absorbing Lotion,La Mer,"[Algae Extract, Butylene Glycol, Squalane, Pro...",24
1,4427,The Moisturizing Gel Cream,La Mer,"[Algae Extract, Cyclopentasiloxane, Butylene G...",22
2,2843,The Mist,La Mer,"[Water, Butylene Glycol, Glycerin, Sucrose, Al...",19
3,2703,The Moisturizing Lotion,La Mer,"[Algae Extract, Cyclopentasiloxane, Glyceryl D...",25
4,3947,The Brightening Infusion Intense,La Mer,"[Water, Alcohol Denat., Yeast Extract, Methyl ...",24
5,270,Creme,Nivea,"[Water, Mineral Oil, Petrolatum, Glycerin, Mic...",15
6,4217,The Brightening Lotion Intense,La Mer,"[Water, Yeast Extract, Isopentyldiol, Ascorbyl...",22
7,4380,The Brightening Essence Intense,La Mer,"[Water, Dimethicone, Trisiloxane, Yeast Extrac...",22
8,12571,Brilliance Brightening Lotion Intense,La Mer,"[Water, Yeast Extract, Isopentyldiol, Ascorbyl...",21
9,4534,The Concentrate,La Mer,"[Cyclopentasiloxane, Algae Extract, Glycerin, ...",17


### Find specific product similarity

In [461]:
prod_search_id = 13566
sims = [v for i, v in enumerate(sim_scores_all) if v[0] == prod_search_id]

sim_df = pd.DataFrame(sim_scores_all, columns=['index', 'score']).sort_values(['score'], ascending = False).reset_index().drop(['level_0'], axis = 1)
index_return = sim_df[sim_df['index'] == prod_search_id].index.item() 

list1 = df.loc[idx]['ingList']
list2 = df.loc[prod_search_id]['ingList']

print('Cosine Similarity')
print('---------------------------------------------------------------')
print('Product:', df.iloc[prod_search_id]['product'])
print('Similarity score:', sims[0][1])
print('Rank:', index_return)
print('Number of Ingredients', df.iloc[prod_search_id]['ingCount'])
print('Number of Common Ingredients', len(set(list1)&set(list2)))
print('Ingredients', df.iloc[prod_search_id]['ingList'])

Cosine Similarity
---------------------------------------------------------------
Product: Black Sea Deep Hydration Cream
Similarity score: 0.312170192438
Rank: 18
Number of Ingredients 46
Number of Common Ingredients 20
Ingredients ['Mineral Oil', 'Hydrogenated Vegetable Oil', 'Isohexadecane', 'Microcrystalline Wax', 'Glycerin', 'Cocoglycerides', 'Fragrance', 'Paraffin', 'Cetearyl Alcohol', 'Aluminum Tristearate', 'Panthenol', 'Corylus Avellana (Hazel) Seed Oil', 'Magnesium Aspartate', 'Zinc Gluconate', 'Copper Gluconate', 'Phospholipids', 'Retinyl Palmitate (Vitamin A', 'Tocopherol (Vitamin E', 'Ascorbyl Palmitate', 'Beta-Carotene', 'Magnesium Sulfate', 'Lamium Album Extract', 'Tripleurospermum Maritima Extract', 'Hydrolyzed Pea Protein', 'Lecithin', 'Olea Europaea (Olive) Fruit Oil', 'Phytosterols', 'Squalane', 'Butyrospermum Parkii (Shea) Butter', 'Ceramide 3', 'SD Alcohol 40', 'Polysorbate 60', 'Propylene Glycol', 'Water', 'Benzyl Benzoate', 'Benzyl Salicylate', 'Citral', 'Citrone

## RBO similarity measure

Based on code found [here](https://github.com/dlukes/rbo/blob/master/rbo.py)

### Create dataframe of top ten similar products

In [462]:
%%time

df = dataPrep(ings, prods, prod_ing)
##Set the rbo we want to use
rbo = rbo_min

itemLookup = df.loc[idx]['ing#List']
items = df['ing#List']
sim = [rbo(itemLookup,i, .9) for i in items] ##change method here

sim_scores_all = list(enumerate(sim))

# Sort the products based on the similarity scores
sim_scores_sorted = sorted(sim_scores_all, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores_ten = sim_scores_sorted[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores_ten]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

Wall time: 21.9 s


### Show top ten

In [463]:
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
#recommended.to_csv('recommendations/RBO.csv')
topTen.reset_index()

,index,product,brand,ingList,commonIng
0,4427,The Moisturizing Gel Cream,La Mer,"[Algae Extract, Cyclopentasiloxane, Butylene G...",22
1,2763,Intense Moisturizing Cream,Mary Kay,"[Water, Mineral Oil, Glycerin, Petrolatum, Iso...",10
2,270,Creme,Nivea,"[Water, Mineral Oil, Petrolatum, Glycerin, Mic...",15
3,3581,The Moisturizing Soft Cream,La Mer,"[Algae Extract, Cyclopentasiloxane, Petrolatum...",23
4,3671,Intensive Healing Cream Daily Fragrance-Free C...,Curel,"[Water, Mineral Oil, Petrolatum, Glycerin, Mic...",7
5,855,Dramatically Different Moisturizing Lotion+ wi...,Clinique,"[Water, Mineral Oil, Glycerin, Petrolatum, Ste...",6
6,2927,The Oil Absorbing Lotion,La Mer,"[Algae Extract, Butylene Glycol, Squalane, Pro...",24
7,2735,The Eye Concentrate,La Mer,"[Algae Extract, Cyclopentasiloxane, Glycerin, ...",17
8,2703,The Moisturizing Lotion,La Mer,"[Algae Extract, Cyclopentasiloxane, Glyceryl D...",25
9,13566,Black Sea Deep Hydration Cream,Edward Bess,"[Mineral Oil, Hydrogenated Vegetable Oil, Isoh...",20


### Find specific product similarity

In [470]:
prod_search_id = 13566
sims = [v for i, v in enumerate(sim_scores_all) if v[0] == prod_search_id]

sim_df = pd.DataFrame(sim_scores_all, columns=['index', 'score']).sort_values(['score'], ascending = False).reset_index().drop(['level_0'], axis = 1)
index_return = sim_df[sim_df['index'] == prod_search_id].index.item() 

list1 = df.loc[idx]['ingList']
list2 = df.loc[prod_search_id]['ingList']

print('RBO Similarity')
print('---------------------------------------------------------------')
print('Product:', df.iloc[prod_search_id]['product'])
print('Similarity score:', sims[0][1])
print('Rank:', index_return)
#print('Number of Ingredients', df.iloc[prod_search_id]['ingCount'])
#print('Number of Common Ingredients', len(set(list1)&set(list2)))
#print('Ingredients', df.iloc[prod_search_id]['ingList'])

RBO Similarity
---------------------------------------------------------------
Product: Black Sea Deep Hydration Cream
Similarity score: 0.3551992038899179
Rank: 10


## R value
Reference [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.linregress.html)

In [471]:
%%time
#get linregress similarity and return r-value
sim = [linregress(df2[idx], df2[i])[2] for i in df2]

##collect all similarity scores (r_val)
sim_scores_all = list(enumerate(sim))

# Sort the products based on the similarity scores
sim_scores_sorted = sorted(sim_scores_all, key=lambda x: x[1], reverse=True)
#df['scores'] = [sim_scores[i][1] for i in sim_scores]

# Get the scores of the 10 most similar products
sim_scores_ten = sim_scores_sorted[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores_ten]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

Wall time: 1min 1s


### Return top ten

In [472]:
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
#recommended.to_csv('recommendations/RBO.csv')
topTen.reset_index()

,index,product,brand,ingList,commonIng
0,270,Creme,Nivea,"[Water, Mineral Oil, Petrolatum, Glycerin, Mic...",15
1,2927,The Oil Absorbing Lotion,La Mer,"[Algae Extract, Butylene Glycol, Squalane, Pro...",24
2,4427,The Moisturizing Gel Cream,La Mer,"[Algae Extract, Cyclopentasiloxane, Butylene G...",22
3,2703,The Moisturizing Lotion,La Mer,"[Algae Extract, Cyclopentasiloxane, Glyceryl D...",25
4,2843,The Mist,La Mer,"[Water, Butylene Glycol, Glycerin, Sucrose, Al...",19
5,3947,The Brightening Infusion Intense,La Mer,"[Water, Alcohol Denat., Yeast Extract, Methyl ...",24
6,4217,The Brightening Lotion Intense,La Mer,"[Water, Yeast Extract, Isopentyldiol, Ascorbyl...",22
7,12571,Brilliance Brightening Lotion Intense,La Mer,"[Water, Yeast Extract, Isopentyldiol, Ascorbyl...",21
8,13566,Black Sea Deep Hydration Cream,Edward Bess,"[Mineral Oil, Hydrogenated Vegetable Oil, Isoh...",20
9,2922,The Cleansing Gel,La Mer,"[Cocamidopropyl Hydroxysultaine, Sodium Cocoyl...",18


### Find specific product similarity

In [481]:
prod_search_id = 13566
sims = [v for i, v in enumerate(sim_scores_all) if v[0] == prod_search_id]

sim_df = pd.DataFrame(sim_scores_all, columns=['index', 'score']).sort_values(['score'], ascending = False).reset_index().drop(['level_0'], axis = 1)
index_return = sim_df[sim_df['index'] == prod_search_id].index.item() 

list1 = df.loc[idx]['ingList']
list2 = df.loc[prod_search_id]['ingList']

print('RVal Similarity')
print('---------------------------------------------------------------')
print('Product:', df.iloc[prod_search_id]['product'])
print('Similarity score:', sims[0][1])
print('Rank:', index_return)
#print('Number of Ingredients', df.iloc[prod_search_id]['ingCount'])
#print('Number of Common Ingredients', len(set(list1)&set(list2)))
#print('Ingredients', df.iloc[prod_search_id]['ingList'])

RVal Similarity
---------------------------------------------------------------
Product: Black Sea Deep Hydration Cream
Similarity score: 0.42389064933
Rank: 9
